In [ ]:
# Make sure these libraries are available within your python environement.
# Uncomment the following lines to install the libraries
!pip install --upgrade pip
!pip install 'sagemaker>=2.42.0'
!pip install jsonlines
!pip install pillow
!pip install matplotlib
!pip install 'sagemaker[local]' --upgrade

## 1) Deps and sagemaker env

First requirements :

Make sure you have an AWS IAM Role capable of running SageMaker job, and having read/write access to the S3 buckets thats contains :

  - yolo pretraining model and inputs (see 3. below)
  - data set images and labels
  

In [ ]:
import os
from urllib.parse import urlparse
import io
import boto3
import json
import jsonlines
import sagemaker
import pandas as pd
import numpy as np
from PIL import Image
from itertools import cycle, islice
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sagemaker.estimator import Framework, Estimator
from sagemaker.processing import Processor, ProcessingInput, ProcessingOutput

sagemaker_session = sagemaker.Session()

# we are using the notebook instance role for training in this example
role = 'AmazonSageMaker-ExecutionRole-<YOUR_IAM_EXECUTION_ROLE_ID>' 

# you can specify a bucket name here, we're using the default bucket of SageMaker
bucket = sagemaker_session.default_bucket()

print(role)
print(bucket)


## 2) Download yolo model with pretrained weights

In this example you can either use yolov5 small or large. But you can download other flavor from the yolov5 project, and adjust the training inputs accordingly

In [ ]:
!wget https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5s.pt -O ./yolo-inputs/input/data/weights/yolov5s.pt
!wget https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5l.pt -O ./yolo-inputs/input/data/weights/yolov5l.pt

## 3) Archive yolo configuration items and push them to S3

### Job configuration 

in local folder 'yolo-inputs' :

  - make sure that in input/data/weights you can find the preweighted model you want to use for the transfer learning
  - make sure that in input/data/cfg you can find the yolo model configuration file related to the preweighted model you want to use. If not file are available here : https://github.com/ultralytics/yolov5/tree/master/models
  - make sure that in input/data/cfg you can find the yolo model hyperparameter file. If not file are available here : https://github.com/ultralytics/yolov5/tree/master/data/hyps
  - in input/data/cfg/train-args.json adjust the training job parameters (number of epochs, batch size, preweighted model name)
  - in input/data/cfg/visualsearch.yaml adjust the number of object categories and their name (nc and names). train and val path must remain as they are : docker volume are mounted to these path.
  
### Job configuration input upload

You now need to upload the content of 'yolo-input' into one of your S3, within a folder structure named 'visualsearch/training-inputs'



## 4) Build yolov5 container. 

First we build our custom yolo v5 docker image, using this command from a bash terminal : 

    AWS_PROFILE=your-aws-profile-name ./build-and-push.sh visualsearch-yolov5l-train

Then we retrieve the docker image id, to be use by the training job

In [ ]:
with open (os.path.join('container', 'ecr_image_fullname.txt'), 'r') as f:
    container = f.readlines()[0][:-1]

print(container)

## 5) Training job configuration

In [ ]:
s3_input = 's3://{}/visualsearch/training-inputs'.format(bucket)
s3_images = "s3://<YOUR DATA SET S3 BUCKET>/visualsearch/dataset" # Images files are here, in a subfolder named 'train'
s3_labels = "s3://<YOUR DATA SET S3 BUCKET>/visualsearch/labels" # Label files are here, in a subfolder named 'train'
print(s3_input)
print(s3_images)
print(s3_labels)

# cfg  images weights labels
cfg='{}/input/data/cfg/'.format(s3_input)
weights='{}/input/data/weights/'.format(s3_input)
outpath='{}/results/'.format(s3_input)

images='{}/'.format(s3_images)
labels='{}/'.format(s3_labels)

print(cfg)
print(weights)
print(outpath)

print(images)
print(labels)

## 6) Run training job

To test the training locally (require docker and docker-compose locally) you can set the 'instance_type' parameter to 'local', otherwise set a EC2 instance type name, to select the required resources (disk, CPU/GPU, Ram)


In [ ]:
# Job configuration
from sagemaker.session import TrainingInput

inputs = {
    "cfg": TrainingInput(cfg),
    "images": TrainingInput(images),
    "weights": TrainingInput(weights),
    "labels": TrainingInput(labels),
}

estimator = Estimator(
    image_uri=container,
    role=role,
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    # instance_type='local',
    input_mode='File',
    output_path=outpath,
    base_job_name='visualsearch-yolov5'
)



In [ ]:
# To start the job
estimator.fit(inputs)